In [1]:
import numpy as np
import itertools as itr
import matplotlib.pyplot as plt
from time import sleep

from torch import Tensor, manual_seed, is_tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS, Adam
from torch.utils.data import Dataset, DataLoader

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile, assemble, IBMQ
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.opflow import AerPauliExpectation
from qiskit.circuit import Parameter,ParameterVector
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, ZZFeatureMap
from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian
#from sklearn.preprocessing import OneHotEncoder
from qiskit.algorithms.optimizers import SPSA, GradientDescent, QNSPSA, ADAM
#from qiskit.tools.monitor import job_monitor


from qiskit_machine_learning.runtime import TorchRuntimeClient, TorchRuntimeResult
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.circuit.library import RawFeatureVector

COUNTER = 0

#Amp Encoding
#DIR_val_train = "Encode_data/amp_enc_data_set_trainning_values.csv"
#DIR_cls_train = "Encode_data/amp_enc_data_set_trainning_classes.csv"
#DIR_val_test = "Encode_data/amp_enc_data_set_test_values.csv"
#DIR_cls_test = "Encode_data/amp_enc_data_set_test_classes.csv"

#Enc Encoding
DIR_val_train = "Encode_data/ang_enc_data_set_trainning_values.csv"
DIR_cls_train = "Encode_data/ang_enc_data_set_trainning_classes.csv"
DIR_val_test = "Encode_data/ang_enc_data_set_test_values.csv"
DIR_cls_test = "Encode_data/ang_enc_data_set_test_classes.csv"

#IBMQ.save_account("", overwrite=True)

IBMQ.enable_account("dcbfead05af57561bd8ede02e3ba82d103c40683442fd5742116c4cc079999d49e865bc6b579a2f21186e8b7fb732c7161712ff6f41fcd9ea4c820e6d4d53d00")

provider = IBMQ.get_provider(hub="ibm-q")  # replace by your runtime provider
backend = provider.get_backend("ibmq_manila")  # select a backend that supports the runtime

# Create Data Set/Loader

In [2]:
def get_encode(file):
    """
     Funcao responsavel pelo encoding (amplitude)
        Input :: 
        ### file : File dir
        Output :: 
        #### data_enc : Valores para o encode (numpy array)
    """
    return np.genfromtxt(file, delimiter=";")

# Train data points
train_data = get_encode(DIR_val_train)
train_data_T = Tensor(train_data)
train_labels = np.genfromtxt(DIR_cls_train, delimiter=";")
train_labels_T = Tensor(train_labels).reshape(len(train_labels),1)

# Test data points
test_data = get_encode(DIR_val_test)
test_data_T = Tensor(test_data)
test_labels = np.genfromtxt(DIR_cls_test, delimiter=";")
test_labels_T = Tensor(test_labels).reshape(len(test_labels),1)

In [3]:
# Create custom torch dataset class
class TorchDataset(Dataset):
    """Map-style dataset"""

    def __init__(self, X, y):
        self.X = Tensor(X)
        self.y = Tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        import torch

        if torch.is_tensor(idx):
            idx = idx.tolist()

        X_i = self.X[idx]
        y_i = self.y[idx]

        # important: the dataset item must be returned as data,target
        return X_i, y_i

In [4]:
train_set = TorchDataset(train_data_T, train_labels_T)
train_loader = DataLoader(train_set, batch_size=1, shuffle=False)
test_set = TorchDataset(test_data_T, test_labels_T)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

# Create Circuit and Torch Model

In [5]:
# Cicrcuito
n_qubits = 5

encoding = ZZFeatureMap(feature_dimension=n_qubits, reps=2)

ansatz = EfficientSU2(n_qubits, 
                      entanglement='full', 
                      reps=3, 
                      insert_barriers=True, 
                      name="U(\u03B8)", 
                      parameter_prefix="\u03B8")

# Moderl 1
#qnn1 = TwoLayerQNN(n_qubits, 
#                   encoding, 
#                   ansatz)

# Moderl 2
qc = QuantumCircuit(n_qubits)
qc.append(encoding, range(n_qubits))
qc.append(ansatz, range(n_qubits))
#qc.decompose().draw(output="text")

parity = lambda x: "{:b}".format(x).count("1") % 2
output_shape = 2  
qnn2 = CircuitQNN(circuit=qc,
                input_params=encoding.parameters,
                weight_params=ansatz.parameters,
                interpret=parity,
                output_shape=output_shape,
                sampling=True,
                input_gradients=True)

initial_parameters = np.random.random(ansatz.num_parameters)
#model1 = TorchConnector(qnn1, initial_parameters)
model2 = TorchConnector(qnn2, initial_parameters)

#optimizer1 = Adam(model1.parameters(), lr=0.1)
#loss_func1 = MSELoss(reduction="sum")

optimizer2 = Adam(model2.parameters(), lr=0.1)
loss_func2 = CrossEntropyLoss(reduction="sum")

In [6]:
def score_func(out, target):
    from numpy import sign
    score = 0
    if sign(out.item()) == target.item():
        score = 1
    return score

In [7]:
%%script echo skipping
def score_func(out, target):
    from numpy import sign
    score = 0
    p = out.item()
    output = 0 if p[0] > p[1] else 1 
    
    if output == target.item():
        score = 1
    return score

skipping


# Trainning

In [ ]:
torch_runtime_client_QNN_CEL = TorchRuntimeClient(
    provider=provider,
    model=model2,
    optimizer=optimizer2,
    loss_func=loss_func2,
    epochs=2,
    backend=backend)
    
fit_result_TW_MSE = torch_runtime_client_QNN_CEL.fit(train_loader=train_loader)
print("train execution time: ", fit_result_TW_MSE.execution_time)

score_result = torch_runtime_client_QNN_CEL.score(data_loader=test_loader, score_func=score_func)
print("id: ", score_result.job_id)
print("execution time: ", score_result.execution_time)
print("score: ", score_result.score)